In [1]:
import albumentations as A
import cv2
import keras
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from tqdm.notebook import tqdm, trange
import segmentation_models as sm
from time import gmtime, strftime
import glob


"""
segmentation_models could also use `tf.keras` if you do not have Keras installed 
or you could switch to other framework using `sm.set_framework('tf.keras')`
"""
sm.set_framework('tf.keras')

# Global Variable
DATA_DIR = "/ich/ICH-Segmentation/datasets/ICH_420/export/Positive"
OUTPUT_DIR = "/ich/ICH-Segmentation/output"
LOGS_DIR = "/ich/ICH-Segmentation/logs"
TRAINING_NAME = "ICH"
BACKBONE = "efficientnetb3"
BATCH_SIZE = 12
CLASSES = ["ich"]
LR = 0.0001
EPOCHS = 1
START_TIME = strftime('%Y%m%d%H%M%S', gmtime())
MODEL_NAME = f"ICH-{START_TIME}-"+"{epoch}.h5"
MODEL_FOLDER = os.path.join(LOGS_DIR, f"{TRAINING_NAME}-{START_TIME}")
os.makedirs(OUTPUT_DIR, exist_ok=True)

2022-11-08 02:26:24.578152: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-08 02:26:24.641111: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-08 02:26:24.658212: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Segmentation Models: using `keras` framework.


In [2]:
# helper function for data visualization
def visualize(et="", fn=f"tmp-{''.join(random.sample('zyxwvutsrqponmlkjihgfedcba0123456789',6))}.png", **images):
    """PLot images in one row."""
    num_of_images = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, num_of_images, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
        # plt.show()
        if i == 2:
            plt.savefig(os.path.join(OUTPUT_DIR, f"{name}-{i}-{et}-{fn}"))
            plt.close()

In [3]:
# helper function for data visualization
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x

In [4]:
def round_clip_0_1(x, **kwargs):
    return x.round().clip(0, 1)

In [5]:
def display_form_dataset(img_dir: str, annot_dir: str, classes: [str], name="", img_index=0, augmentation=None):
    # Let's look at data we have
    _dataset = Dataset(
        img_dir,
        annot_dir,
        classes=classes,
        augmentation=augmentation
    )

    # get some sample
    _img, _mask = _dataset[img_index]
    visualize(
        # fn=name,
        image=_img,
        ich_mask=_mask[..., 0].squeeze(),
    )

In [6]:
def get_training_parameters():
    # # case for binary and multiclass segmentation
    # Multiclass
    n_classes = len(CLASSES) + 1
    if len(CLASSES) == 1:
        # Binary
        n_classes = 1

    # Activation Type
    activation = 'softmax'
    if n_classes == 1:
        activation = "sigmoid"

    # Create Model
    model = sm.Unet(
        BACKBONE,
        classes=n_classes,
        activation=activation
    )

    return model, n_classes, activation

In [7]:
# classes for data loading and preprocessing
class Dataset:
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.

    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing
            (e.g. noralization, shape manipulation, etc.)

    """

    # CLASSES = ['sky', 'building', 'pole', 'road', 'pavement',
    #            'tree', 'signsymbol', 'fence', 'car',
    #            'pedestrian', 'bicyclist', 'unlabelled']
    CLASSES = ['ich']

    def __init__(
            self,
            images_dir,
            masks_dir,
            classes=None,
            augmentation=None,
            preprocessing=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]

        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]

        self.augmentation = augmentation
        self.preprocessing = preprocessing

    def get_basename(self, i):
        return os.path.basename(self.images_fps[i])

    def __getitem__(self, i):

        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i], 0)

        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')

        # add background if mask is not binary
        if mask.shape[-1] != 1:
            background = 1 - mask.sum(axis=-1, keepdims=True)
            mask = np.concatenate((mask, background), axis=-1)

        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        return image, mask

    def __len__(self):
        return len(self.ids)

In [8]:
class Dataloader(keras.utils.Sequence):
    """Load data from dataset and form batches

    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """

    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):

        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        # transpose list of lists
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]

        return batch

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)

In [9]:
def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        A.PadIfNeeded(512, 512)
    ]
    return A.Compose(test_transform)

In [10]:
def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform

    Args:
        preprocessing_fn (callbale): data normalization function
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose

    """

    _transform = [
        A.Lambda(image=preprocessing_fn),
    ]
    return A.Compose(_transform)

In [11]:
def inference():
    """ Test Dataset """
    x_test_dir = os.path.join(DATA_DIR, "test", "image")
    y_test_dir = os.path.join(DATA_DIR, "test", "mask")

    """ Get Backbone preprocessing """
    preprocess_input = sm.get_preprocessing(BACKBONE)

    test_dataset = Dataset(
        x_test_dir,
        y_test_dir,
        classes=CLASSES,
        preprocessing=get_preprocessing(preprocess_input),
    )

    datasets = [test_dataset]

    test_dataloader = Dataloader(test_dataset, batch_size=1, shuffle=False)
    dataloaders = [test_dataloader]

    """ define network parameters """
    model, n_classes, _ = get_training_parameters()

    # define optimizer
    optim = keras.optimizers.Adam(LR)

    # Segmentation models losses can be combined together by '+' and scaled by integer or float factor
    dice_loss = sm.losses.DiceLoss()
    focal_loss = sm.losses.CategoricalFocalLoss()
    if n_classes == 1:
        sm.losses.BinaryFocalLoss()

    total_loss = dice_loss + (1 * focal_loss)

    # actually total_loss can be imported directly from library,
    # above example just show you how to manipulate with losses
    # total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss
    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

    # compile keras model with defined optimizer, loss and metrics
    model.compile(optim, total_loss, metrics)

    # load best weights
    model.load_weights(os.path.join(MODEL_FOLDER, MODEL_NAME))

    evaluate_type = ["test"]
    for idx, dataloader in enumerate(dataloaders):
        scores = model.evaluate(dataloader)

        print(f"Start process {evaluate_type[idx]}")

        # actually total_loss can be imported directly from library,
        # above example just show you how to manipulate with losses
        # total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss
        metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

        print("Loss: {:.5}".format(scores[0]))
        for metric, value in zip(metrics, scores[1:]):
            print("mean {}: {:.5}".format(metric.__name__, value))

        # n = 5
        # ids = np.random.choice(np.arange(len(test_dataset)), size=n)

        for i in trange(len(datasets[idx]), desc="Predict"):
            image, gt_mask = datasets[idx][i]
            image = np.expand_dims(image, axis=0)
            pr_mask = model.predict(image).round()

            visualize(
                et=evaluate_type[idx],
                fn=datasets[idx].get_basename(i),
                image=denormalize(image.squeeze()),
                gt_mask=gt_mask[..., 0].squeeze(),
                pr_mask=pr_mask[..., 0].squeeze(),
            )

In [12]:
def train():
    os.makedirs(LOGS_DIR, exist_ok=True)
    os.makedirs(MODEL_FOLDER, exist_ok=True)

    """ Train Dataset """
    x_train_dir = os.path.join(DATA_DIR, "train", "image")
    y_train_dir = os.path.join(DATA_DIR, "train", "mask")

    """ Valid Dataset """
    x_valid_dir = os.path.join(DATA_DIR, "val", "image")
    y_valid_dir = os.path.join(DATA_DIR, "val", "mask")

    """ Show Dataset """
    # display_form_dataset(x_train_dir,
    #                      y_train_dir,
    #                      img_index=5,
    #                      classes=['car', 'pedestrian']
    #                      )

    """ Show Augmentation Dataset """
    # Lets look at augmented data we have
    # display_form_dataset(x_train_dir,
    #                      y_train_dir,
    #                      img_index=12,
    #                      classes=["ich"],
    #                      augmentation=get_training_augmentation()
    #                      )

    """ Get Backbone preprocessing """
    preprocess_input = sm.get_preprocessing(BACKBONE)

    """ define network parameters """
    model, n_classes, _ = get_training_parameters()
    
    # define optimizer
    optim = keras.optimizers.Adam(LR)

    # Segmentation models losses can be combined together by '+' and scaled by integer or float factor
    dice_loss = sm.losses.DiceLoss()
    focal_loss = sm.losses.CategoricalFocalLoss()
    if n_classes == 1:
        sm.losses.BinaryFocalLoss()

    total_loss = dice_loss + (1 * focal_loss)

    # actually total_loss can be imported directly from library,
    # above example just show you how to manipulate with losses
    # total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss
    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

    # compile keras model with defined optimizer, loss and metrics
    model.compile(optim, total_loss, metrics)

    # Dataset for train images
    train_dataset = Dataset(
        x_train_dir,
        y_train_dir,
        classes=CLASSES,
        augmentation=get_training_augmentation(),
        preprocessing=get_preprocessing(preprocess_input),
    )

    # Dataset for validation images
    valid_dataset = Dataset(
        x_valid_dir,
        y_valid_dir,
        classes=CLASSES,
        augmentation=get_validation_augmentation(),
        preprocessing=get_preprocessing(preprocess_input),
    )

    train_dataloader = Dataloader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_dataloader = Dataloader(valid_dataset, batch_size=1, shuffle=True)

    # check shapes for errors
    assert train_dataloader[0][0].shape == (BATCH_SIZE, 512, 512, 3)
    assert train_dataloader[0][1].shape == (BATCH_SIZE, 512, 512, n_classes)

    # define callbacks for learning rate scheduling and best checkpoints saving
    callbacks = [
        keras.callbacks.TensorBoard(log_dir=MODEL_FOLDER),
        keras.callbacks.ModelCheckpoint(os.path.join(MODEL_FOLDER, MODEL_NAME), save_weights_only=True, save_best_only=False, mode='min'),
        keras.callbacks.ReduceLROnPlateau(),
    ]

    # train model
    history = model.fit(
        train_dataloader,
        steps_per_epoch=len(train_dataloader),
        epochs=EPOCHS,
        callbacks=callbacks,
        validation_data=valid_dataloader,
        validation_steps=len(valid_dataloader),
    )

    # Plot training & validation iou_score values
    plt.figure(figsize=(30, 5))
    plt.subplot(121)
    plt.plot(history.history['iou_score'])
    plt.plot(history.history['val_iou_score'])
    plt.title('Model iou_score')
    plt.ylabel('iou_score')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(122)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.savefig(os.path.join(MODEL_FOLDER, "training.png"))

In [13]:
# history.history

In [14]:
# define heavy augmentations
def get_training_augmentation():
    train_transform = [
        A.Flip(p=0.5),
        A.Rotate(limit=[90, -90], border_mode=0, p=0.5),
        A.OneOf([
            A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.5, p=0.5),
            A.CLAHE(p=0.5),  
            A.Equalize(p=0.5),
            A.RandomGamma(gamma_limit=[60, 150], p=0.5),
        ]),
        A.OneOf([
            A.ISONoise(color_shift=[0.01, 0.05], intensity=[0.6, 0.6], p=0.5),
            A.GaussNoise(var_limit=150, p=0.5),
            A.ImageCompression(quality_lower=80, quality_upper=100, p=0.5),
        ]),
        A.InvertImg(p=0.3),
        A.Sharpen(p=0.5),
        A.Lambda(mask=round_clip_0_1)
    ]
    return A.Compose(train_transform)

In [15]:
def detect(dataset_path, model_filepath, name):
    """ Test Dataset """
    x_test_dir = os.path.join(dataset_path, "image")
    y_test_dir = os.path.join(dataset_path, "mask")

    """ Get Backbone preprocessing """
    preprocess_input = sm.get_preprocessing(BACKBONE)

    test_dataset = Dataset(
        x_test_dir,
        y_test_dir,
        classes=CLASSES,
        preprocessing=get_preprocessing(preprocess_input),
    )
    datasets = [test_dataset]

    test_dataloader = Dataloader(test_dataset, batch_size=1, shuffle=True)
    dataloaders = [test_dataloader]

    """ define network parameters """
    model, n_classes, _ = get_training_parameters()

    # define optimizer
    optim = keras.optimizers.Adam(LR)

    # Segmentation models losses can be combined together by '+' and scaled by integer or float factor
    dice_loss = sm.losses.DiceLoss()
    focal_loss = sm.losses.CategoricalFocalLoss()
    if n_classes == 1:
        sm.losses.BinaryFocalLoss()

    total_loss = dice_loss + (1 * focal_loss)

    # actually total_loss can be imported directly from library,
    # above example just show you how to manipulate with losses
    # total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss
    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

    # compile keras model with defined optimizer, loss and metrics
    model.compile(optim, total_loss, metrics)

    # load best weights
    model.load_weights(model_filepath)

    evaluate_type = ["test", "val"]
    for idx, dataloader in enumerate(dataloaders):
        scores = model.evaluate(dataloader)

        print(f"Start process {evaluate_type[idx]}")

        # actually total_loss can be imported directly from library,
        # above example just show you how to manipulate with losses
        # total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss
        metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

        print("Loss: {:.5}".format(scores[0]))
        for metric, value in zip(metrics, scores[1:]):
            print("mean {}: {:.5}".format(metric.__name__, value))

        # for i in trange(len(datasets[idx]), desc="Predict"):
        for i in trange(50, desc="Predict"):
            image, gt_mask = datasets[idx][i]
            image = np.expand_dims(image, axis=0)
            pr_mask = model.predict(image).round()
            
            visualize(
                et=name,
                fn=datasets[idx].get_basename(i),
                image=denormalize(image.squeeze()),
                gt_mask=gt_mask[..., 0].squeeze(),
                pr_mask=pr_mask[..., 0].squeeze(),
            )

In [16]:
def main():
    # train()
    # inference()
    detect(
        # Dataset
        # "/ich/ICH-Segmentation/datasets/ICH_420/export/test",
        "/workspaces/Intracranial-Hemorrhage/ICH-Segmentation/datasets/ICH_420/export/Positive/test",

        # Prob
        # "/ich/ICH-Segmentation/datasets/ICH_127",
        
        # Model
        # "/ich/ICH-Segmentation/logs/ICH-20221106214837/ICH-20221106214837-30.h5",
        # "/ich/ICH-Segmentation/logs/ICH-20221107052020/ICH-20221107052020-10.h5",
        # "/ich/ICH-Segmentation/logs/ICH-20221106182510/ICH-20221106182510-model.h5",
        "/workspaces/Intracranial-Hemorrhage/ICH-Segmentation/logs/ICH420-20221107165659/ICH-ICH420-31.h5",
        
        # Name
        "ICH127"
    )

In [17]:
if __name__ == '__main__':
    main()

FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/Intracranial-Hemorrhage/ICH-Segmentation/datasets/ICH_420/export/Positive/test/image'